In [2]:
import numpy as np
import pandas as pd

In [3]:
theme_df = pd.read_csv("./data/theme_df.csv", index_col=0)
vote_df = pd.read_csv("./data/vote_df.csv", index_col=0)

In [4]:
theme_df["theme"] = theme_df["loc_1"] + "_" + theme_df["theme"]

In [5]:
vote_df.drop(columns=["etc"], inplace=True)
vote_theme_df = vote_df.merge(theme_df, on=["loc_1", "loc_2", "loc_3"])

vote_theme_gb_df = vote_theme_df.groupby(["theme", "time"])[["lib", "con"]].sum().reset_index()
vote_theme_gb_df.sort_values("time", inplace=True)

In [6]:
def weighted_mean(values):
    return np.average(values, weights=np.exp(range(1, len(values) + 1)))

In [7]:
_weighted_mean_df = lambda col: vote_theme_gb_df.groupby("theme")[col].apply(weighted_mean)

weighted_mean_vote_df = pd.concat(list(map(_weighted_mean_df, ["lib", "con"])), axis=1)
weighted_mean_vote_df

,lib,con
theme,,
강원도_강릉시,36673.418394,51085.348800
강원도_동해시태백시삼척시정선군,46637.209255,64438.870930
강원도_속초시인제군고성군양양군,35634.315792,45858.502775
강원도_원주시갑,36026.700427,40899.916063
강원도_원주시을,34734.743859,36661.093119
...,...,...
충청북도_청주시상당구,34497.974706,42230.004173
충청북도_청주시서원구,35879.658617,43350.512387
충청북도_청주시청원구,35231.656653,37442.050348


In [8]:
wmv_df = weighted_mean_vote_df.copy()

In [9]:
wmv_df["lib_rat"] = wmv_df["lib"] / (wmv_df["lib"] + wmv_df["con"])
wmv_df.drop(columns=["lib", "con"], inplace=True)

In [10]:
wmv_df

,lib_rat
theme,
강원도_강릉시,0.417889
강원도_동해시태백시삼척시정선군,0.419867
강원도_속초시인제군고성군양양군,0.437269
강원도_원주시갑,0.468326
강원도_원주시을,0.486509
...,...
충청북도_청주시상당구,0.449614
충청북도_청주시서원구,0.452853
충청북도_청주시청원구,0.484792


In [11]:
vote_ratio_df = pd.read_csv("./data/vote_ratio_df.csv", index_col=0)

In [12]:
vote_ratio_theme_df = vote_ratio_df.merge(
    theme_df.loc[:, ["loc_1", "loc_2", "theme"]], on=["loc_1", "loc_2"]
)
vote_ratio_theme_df

,loc_1,loc_2,arg_1,18_29,30_39,40_49,50_59,60_,time,theme
0,서울특별시,중구,남자,9.3,10.3,8.0,8.8,12.2,2020_04,서울특별시_중구성동구을
1,서울특별시,중구,남자,9.3,10.3,8.0,8.8,12.2,2020_04,서울특별시_중구성동구을
2,서울특별시,중구,남자,9.3,10.3,8.0,8.8,12.2,2020_04,서울특별시_중구성동구을
3,서울특별시,중구,남자,9.3,10.3,8.0,8.8,12.2,2020_04,서울특별시_중구성동구을
4,서울특별시,중구,남자,9.3,10.3,8.0,8.8,12.2,2020_04,서울특별시_중구성동구을
...,...,...,...,...,...,...,...,...,...,...
20327,경상남도,진주시,여자,7.2,4.7,6.5,8.9,22.8,2022_06,경상남도_진주시을
20328,경상남도,진주시,여자,7.2,4.7,6.5,8.9,22.8,2022_06,경상남도_진주시을
20329,경상남도,진주시,여자,7.2,4.7,6.5,8.9,22.8,2022_06,경상남도_진주시을
20330,경상남도,진주시,여자,7.2,4.7,6.5,8.9,22.8,2022_06,경상남도_진주시을


In [13]:
vote_ratio_theme_df.drop_duplicates(inplace=True)

vote_ratio_theme_gb_df = vote_ratio_theme_df.groupby(["theme", "time"])[
    ["18_29", "30_39", "40_49", "50_59", "60_"]
].mean()

vote_ratio_theme_gb_df = vote_ratio_theme_gb_df.apply(lambda x: x * 2)
vote_ratio_theme_gb_df.reset_index(inplace=True)
vote_ratio_theme_gb_df

tmp = vote_ratio_theme_gb_df.drop(columns=["time"])
cols = tmp.columns[1:]

df_wm = lambda col: tmp.groupby("theme")[col].apply(weighted_mean)

vote_ratio_mean_df = pd.concat(list(map(df_wm, cols)), axis=1)
vote_ratio_mean_df = vote_ratio_mean_df.astype(int)

In [14]:
# 변동성
vote_ratio_mean_df

,18_29,30_39,40_49,50_59,60_
theme,,,,,
강원도_강릉시,14,11,16,19,38
강원도_동해시태백시삼척시정선군,11,7,14,20,46
강원도_속초시인제군고성군양양군,13,11,14,19,39
강원도_원주시갑,16,15,19,20,28
강원도_원주시을,16,15,19,20,28
...,...,...,...,...,...
충청북도_청주시상당구,17,16,18,20,27
충청북도_청주시서원구,18,12,18,20,29
충청북도_청주시청원구,17,18,17,17,27


In [15]:
weights = np.array([0.5, 0.0, 0.0, 0.0, 0.0])

In [16]:
vote_chg_ratio_df = vote_ratio_mean_df.div(vote_ratio_mean_df.sum(axis=1), axis=0).mul(weights)

In [17]:
vote_chg_ratio_df

,18_29,30_39,40_49,50_59,60_
theme,,,,,
강원도_강릉시,0.071429,0.0,0.0,0.0,0.0
강원도_동해시태백시삼척시정선군,0.056122,0.0,0.0,0.0,0.0
강원도_속초시인제군고성군양양군,0.067708,0.0,0.0,0.0,0.0
강원도_원주시갑,0.081633,0.0,0.0,0.0,0.0
강원도_원주시을,0.081633,0.0,0.0,0.0,0.0
...,...,...,...,...,...
충청북도_청주시상당구,0.086735,0.0,0.0,0.0,0.0
충청북도_청주시서원구,0.092784,0.0,0.0,0.0,0.0
충청북도_청주시청원구,0.088542,0.0,0.0,0.0,0.0


In [18]:
wmv_df

,lib_rat
theme,
강원도_강릉시,0.417889
강원도_동해시태백시삼척시정선군,0.419867
강원도_속초시인제군고성군양양군,0.437269
강원도_원주시갑,0.468326
강원도_원주시을,0.486509
...,...
충청북도_청주시상당구,0.449614
충청북도_청주시서원구,0.452853
충청북도_청주시청원구,0.484792


In [19]:
theme_loc_dict = theme_df.set_index("theme")["loc_1"].to_dict()

In [20]:
strong_lib = wmv_df[wmv_df["lib_rat"] > 0.6].index
strong_lib

Index(['광주광역시_광산구갑', '광주광역시_광산구을', '광주광역시_동구남구갑', '광주광역시_동구남구을', '광주광역시_북구갑',
       '광주광역시_북구을', '광주광역시_서구갑', '광주광역시_서구을', '전라남도_고흥군보성군장흥군강진군',
       '전라남도_나주시화순군', '전라남도_담양군함평군영광군장성군', '전라남도_목포시', '전라남도_순천시광양시곡성군구례군갑',
       '전라남도_순천시광양시곡성군구례군을', '전라남도_여수시갑', '전라남도_여수시을', '전라남도_영암군무안군신안군',
       '전라남도_해남군완도군진도군', '전라북도_군산시', '전라북도_김제시부안군', '전라북도_남원시임실군순창군',
       '전라북도_완주군진안군무주군장수군', '전라북도_익산시갑', '전라북도_익산시을', '전라북도_전주시갑', '전라북도_전주시병',
       '전라북도_전주시을', '전라북도_정읍시고창군'],
      dtype='object', name='theme')

In [21]:
strong_con = wmv_df[wmv_df["lib_rat"] < 0.4].index
strong_con

Index(['경상남도_밀양시의령군함안군창녕군', '경상남도_사천시남해군하동군', '경상남도_산청군함양군거창군합천군', '경상남도_양산시갑',
       '경상남도_진주시갑', '경상남도_진주시을', '경상남도_창원시마산합포구', '경상남도_창원시마산회원구',
       '경상남도_창원시의창구', '경상남도_창원시진해구', '경상남도_통영시고성군', '경상북도_경산시', '경상북도_경주시',
       '경상북도_고령군성주군칠곡군', '경상북도_구미시갑', '경상북도_구미시을', '경상북도_군위군의성군청송군영덕군',
       '경상북도_김천시', '경상북도_상주시문경시', '경상북도_안동시예천군', '경상북도_영주시영양군봉화군울진군',
       '경상북도_영천시청도군', '경상북도_포항시남구울릉군', '경상북도_포항시북구', '대구광역시_달서구갑',
       '대구광역시_달서구병', '대구광역시_달서구을', '대구광역시_달성군', '대구광역시_동구갑', '대구광역시_동구을',
       '대구광역시_수성구갑', '대구광역시_수성구을', '대구광역시_중구남구', '부산광역시_금정구', '부산광역시_기장군',
       '부산광역시_남구갑', '부산광역시_남구을', '부산광역시_동래구', '부산광역시_부산진구갑', '부산광역시_부산진구을',
       '부산광역시_사하구갑', '부산광역시_서구동구', '부산광역시_수영구', '부산광역시_연제구', '부산광역시_해운대구갑',
       '부산광역시_해운대구을', '서울특별시_강남구갑', '서울특별시_강남구병', '서울특별시_강남구을', '서울특별시_서초구갑',
       '서울특별시_서초구을', '서울특별시_송파구을', '울산광역시_남구갑', '울산광역시_남구을', '울산광역시_중구',
       '충청남도_홍성군예산군'],
      dtype='object', name='theme')

In [22]:
prediction = weighted_mean_vote_df.copy()

In [23]:
prediction['position'] = ''

con_con = prediction["lib"] < prediction["con"]
lib_con = prediction["con"] < prediction["lib"]

prediction.loc[con_con.values, "position"] = "보수"
prediction.loc[lib_con.values, "position"] = "진보"

In [24]:
print("지역구 국회의원 총수 :", len(prediction))
print("= = " * 10)
print(prediction["position"].value_counts())
print("= = " * 10)
print(prediction["position"].value_counts() / len(prediction))

지역구 국회의원 총수 : 253
= = = = = = = = = = = = = = = = = = = = 
position
보수    144
진보    109
Name: count, dtype: int64
= = = = = = = = = = = = = = = = = = = = 
position
보수    0.56917
진보    0.43083
Name: count, dtype: float64


In [25]:
tmp = pd.merge(prediction, vote_chg_ratio_df["18_29"], left_index=True, right_index=True)

con2lib_18_29 = tmp["18_29"] * tmp["con"] * 0.5

In [26]:
prediction = weighted_mean_vote_df.copy()

In [27]:
prediction['position'] = ''

In [28]:
pred_strong_con = prediction[prediction.index.isin(strong_con)]
pred_strong_con.loc[:, "position"] = "보수"

pred_strong_lib = prediction[prediction.index.isin(strong_lib)]
pred_strong_lib.loc[:, "position"] = "진보"

In [29]:
pred_foggy = prediction[~(prediction.index.isin(strong_con) | prediction.index.isin(strong_lib))].copy()

pred_foggy["lib"] = pred_foggy["lib"] + con2lib_18_29
pred_foggy["con"] = pred_foggy["con"] - con2lib_18_29

con_con = pred_foggy["lib"] < pred_foggy["con"]
lib_con = pred_foggy["con"] < pred_foggy["lib"]

pred_foggy.loc[con_con.values, "position"] = "보수"
pred_foggy.loc[lib_con.values, "position"] = "진보"

In [30]:
prediction = pd.concat([pred_strong_lib, pred_strong_con, pred_foggy], axis=0)

In [31]:
print("지역구 국회의원 총수 :", len(prediction))
print("= = " * 10)
print(prediction["position"].value_counts())
print("= = " * 10)
print(prediction["position"].value_counts() / len(prediction))

지역구 국회의원 총수 : 253
= = = = = = = = = = = = = = = = = = = = 
position
진보    136
보수    117
Name: count, dtype: int64
= = = = = = = = = = = = = = = = = = = = 
position
진보    0.537549
보수    0.462451
Name: count, dtype: float64


In [32]:
prediction.loc[:, ["lib", "con"]] = prediction.loc[:, ["lib", "con"]].div(
    prediction.loc[:, ["lib", "con"]].sum(axis=1), axis=0
)

In [33]:
prediction.columns = ["진보_지지율", "보수_지지율", "승리당"]

In [34]:
prediction.sort_index(inplace=True)

In [41]:
prediction.loc[:,['진보_지지율', '보수_지지율']] = prediction.loc[:,['진보_지지율', '보수_지지율']].apply(lambda x: round(x, 2))

In [43]:
prediction.to_csv("./../prediction.csv")